In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pywt

def preprocess_image(image_path, img_size=(640, 640)):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise FileNotFoundError(f"Image at path {image_path} not found.")

    # Replace all white pixels (value > 250) with black
    image[image > 250] = 0

    original_shape = image.shape

    # Detect and remove white borders using Otsu's thresholding
    _, thresh = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Find all non-white pixels and get their coordinates
    non_white_pixels = np.where(thresh < 255)

    if non_white_pixels[0].size == 0 or non_white_pixels[1].size == 0:
        raise ValueError("No relevant pixels found in the image.")

    # Get the smallest and largest x and y coordinates and use them to create the bounding box
    y_min, y_max = np.min(non_white_pixels[0]), np.max(non_white_pixels[0])
    x_min, x_max = np.min(non_white_pixels[1]), np.max(non_white_pixels[1])

    # Crop the image to the bounding box of all non-white pixels
    image = image[y_min:y_max+1, x_min:x_max+1]

    # Noise reduction using median blur, bilateral filter, and non-local means denoising
    image_median = cv2.medianBlur(image, 5)
    image_bilateral = cv2.bilateralFilter(image_median, 9, 75, 75)
    image_denoised = cv2.fastNlMeansDenoising(image_bilateral, h=30)

    # Wavelet denoising
    coeffs = pywt.wavedec2(image_denoised, 'db1', level=2)
    coeffs[1:] = [tuple(pywt.threshold(i, value=10, mode='soft') for i in level) for level in coeffs[1:]]
    image_wavelet_denoised = pywt.waverec2(coeffs, 'db1')

    # Resize the denoised image
    image_resized = cv2.resize(image_wavelet_denoised, img_size)
    image_resized = np.expand_dims(image_resized, axis=-1)
    image_resized = np.expand_dims(image_resized, axis=0)
    image_resized = image_resized.astype('float32') / 255.0

    return image, image_resized, original_shape, (y_min, y_max, x_min, x_max)

def binary_mask(image):
    # Convert the processed image to uint8
    image_uint8 = (image * 255).astype(np.uint8).squeeze()

    # Apply Otsu's binary thresholding
    _, binary_image = cv2.threshold(image_uint8, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return binary_image

def canny_edge_detection(image):
    edges = cv2.Canny(image, 100, 200)
    return edges

def extract_highest_y(edges):
    height, width = edges.shape
    highest_y_values = np.full(width, height)  # Initialize with maximum Y values (bottom of image)

    for x in range(width):
        column = edges[:, x]
        y_indices = np.where(column > 0)[0]
        if y_indices.size > 0:
            # Sort y_indices to find the highest (smallest) y value that's not 0
            y_indices = np.sort(y_indices)
            for y in y_indices:
                if y != 0:
                    highest_y_values[x] = y
                    break
            else:
                # If all y values are 0, set to the smallest y value (0)
                highest_y_values[x] = y_indices[0]
        else:
            # Handle the case when no edges are found in the column
            if x > 0:
                highest_y_values[x] = highest_y_values[x - 1]
            else:
                highest_y_values[x] = height

    return highest_y_values

def plot_highest_y_on_edges(edges, highest_y_values):
    height, width = edges.shape
    output_image = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

    for x, y in enumerate(highest_y_values):
        if y < height:  # Ensure we don't plot outside the image
            cv2.line(output_image, (x, height), (x, y), (0, 255, 0), 1)  # Plot lines in green

    return output_image

def plot_only_highest_y_values(edges, highest_y_values):
    height, width = edges.shape
    output_image = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)  # Convert edges to a color image

    for x, y in enumerate(highest_y_values):
        if y < height:  # Ensure the Y value is within image bounds
            cv2.circle(output_image, (x, y), 1, (0, 255, 0), -1)  # Draw a small green dot at the highest Y value

    return output_image

def plot_highest_y_on_original(original_image, highest_y_values, scale):
    height, width = original_image.shape
    output_image = cv2.cvtColor(original_image, cv2.COLOR_GRAY2BGR)  # Convert to color image for drawing

    for x, y in enumerate(highest_y_values):
        scaled_x = int(x * scale[1])
        scaled_y = int(y * scale[0])
        if scaled_y < height and scaled_y >= 0 and scaled_x < width and scaled_x >= 0:
            cv2.circle(output_image, (scaled_x, scaled_y), 2, (0, 255, 0), -1)  # Draw a small green dot at the highest Y value

    return output_image

def detect_vertical_gaps(highest_y_values, gap_threshold=5):
    gaps = []
    start_end_points = []
    for x in range(1, len(highest_y_values)):
        if abs(highest_y_values[x] - highest_y_values[x - 1]) > gap_threshold:
            start_end_points.append((x - 1, x))
            gaps.append((x, (highest_y_values[x] + highest_y_values[x - 1]) // 2))
    return gaps, start_end_points

def plot_gaps(output_image, gaps):
    for x, y in gaps:
        cv2.circle(output_image, (x, y), 5, (255, 0, 0), 2)  # Draw a red circle around each gap

    return output_image

def calculate_tangent_vectors(highest_y_values, start_end_points):
    tangent_vectors = []
    for start, end in start_end_points:
        start_point = (start, highest_y_values[start])
        end_point = (end, highest_y_values[end])
        vector = (end_point[0] - start_point[0], end_point[1] - start_point[1])
        tangent_vectors.append((start_point, vector))
    return tangent_vectors

def remove_border_vectors(tangent_vectors, image_width, border_margin=30):
    # Remove rising vectors close to the left border
    tangent_vectors = [vec for vec in tangent_vectors if not (vec[0][0] < border_margin and vec[1][1] < 0)]

    # Remove falling vectors close to the right border
    tangent_vectors = [vec for vec in tangent_vectors if not (vec[0][0] > image_width - border_margin and vec[1][1] > 0)]

    return tangent_vectors

def plot_tangent_vectors(image, tangent_vectors):
    for (start_x, start_y), (vec_x, vec_y) in tangent_vectors:
        end_x = start_x + vec_x
        end_y = start_y + vec_y
        cv2.arrowedLine(image, (start_x, start_y), (end_x, end_y), (255, 0, 255), 2, tipLength=0.2)
    return image

def detect_single_rising_falling_patterns(tangent_vectors):
    patterns = []
    i = 0
    while i < len(tangent_vectors) - 1:
        if tangent_vectors[i][1][1] < 0 and tangent_vectors[i + 1][1][1] > 0:
            patterns.append((tangent_vectors[i], tangent_vectors[i + 1]))
            i += 2  # Move past this pattern
        else:
            i += 1
    return patterns

def draw_pattern_circles(image, patterns, scale=(1.0, 1.0)):
    for rising_vector, falling_vector in patterns:
        x_coords = [rising_vector[0][0], falling_vector[0][0]]
        y_coords = [rising_vector[0][1], falling_vector[0][1]]

        min_x, max_x = min(x_coords), max(x_coords)
        min_y, max_y = min(y_coords), max(y_coords)

        center_x = int((min_x + max_x) // 2 * scale[1])
        center_y = int((min_y + max_y) // 2 * scale[0])
        radius = int(np.sqrt((max_x - min_x) ** 2 + (max_y - min_y) ** 2) / 2 * max(scale))

        cv2.circle(image, (center_x, center_y), radius, (255, 0, 0), 2)
    return image

def process_and_visualize_images(image_paths):
    for image_path in image_paths:
        # Preprocess the image
        original_image, preprocessed_image, original_shape, crop_coords = preprocess_image(image_path)

        # Apply binary masking
        binary_image = binary_mask(preprocessed_image)

        # Apply Canny edge detection
        edges = canny_edge_detection(binary_image)

        # Extract highest Y value edges
        highest_y_values = extract_highest_y(edges)

        # Plot highest Y values on the edges image
        edges_with_highest_y = plot_highest_y_on_edges(edges, highest_y_values)

        # Plot only highest Y values
        highest_y_only = plot_only_highest_y_values(edges, highest_y_values)

        # Plot only highest Y values with dots
        highest_y_with_dots = plot_only_highest_y_values(edges, highest_y_values)

        # Detect vertical gaps and get start-end points
        gaps, start_end_points = detect_vertical_gaps(highest_y_values)

        # Plot gaps on the highest Y values image
        highest_y_with_gaps = plot_gaps(highest_y_only.copy(), gaps)

        # Calculate tangent vectors
        tangent_vectors = calculate_tangent_vectors(highest_y_values, start_end_points)
        print(f"Tangent vectors for {image_path}: {tangent_vectors}")

        # Remove border vectors
        image_width = edges.shape[1]
        tangent_vectors = remove_border_vectors(tangent_vectors, image_width)
        print(f"Tangent vectors after border removal for {image_path}: {tangent_vectors}")

        # Detect single rising and falling patterns
        single_patterns = detect_single_rising_falling_patterns(tangent_vectors)

        print(f"Single rising and falling patterns for {image_path}: {single_patterns}")

        # Draw circles around the detected patterns
        image_with_pattern_circles = draw_pattern_circles(highest_y_only.copy(), single_patterns)

        # Draw circles around the detected patterns on the original image
        scale = (original_shape[0] / 640, original_shape[1] / 640)
        original_with_pattern_circles = draw_pattern_circles(original_image.copy(), single_patterns, scale)

        # Plot highest Y values with dots on the original image
        original_with_dots = plot_highest_y_on_original(original_image.copy(), highest_y_values, scale)

        # Plot tangent vectors on a separate image without circles
        highest_y_with_vectors = plot_tangent_vectors(highest_y_only.copy(), tangent_vectors)

        # Visualize the results
        plt.figure(figsize=(40, 5))
        plt.subplot(1, 10, 1)
        plt.imshow(original_image, cmap='gray')
        plt.title('Original Image')

        plt.subplot(1, 10, 2)
        plt.imshow(binary_image, cmap='gray')
        plt.title('Binary Mask')

        plt.subplot(1, 10, 3)
        plt.imshow(edges, cmap='gray')
        plt.title('Canny Edges')

        plt.subplot(1, 10, 4)
        plt.imshow(edges_with_highest_y)
        plt.title('Highest Y Values on Edges')

        plt.subplot(1, 10, 5)
        plt.imshow(highest_y_with_dots)
        plt.title('Highest Y Values with Dots')

        plt.subplot(1, 10, 6)
        plt.imshow(highest_y_with_gaps)
        plt.title('Only Highest Y Values with Gaps')

        plt.subplot(1, 10, 7)
        plt.imshow(highest_y_with_vectors)
        plt.title('Only Tangent Vectors')

        plt.subplot(1, 10, 8)
        plt.imshow(image_with_pattern_circles)
        plt.title('Rising-Falling Patterns')

        plt.subplot(1, 10, 9)
        plt.imshow(original_with_pattern_circles, cmap='gray')
        plt.title('Original with Patterns')

        plt.subplot(1, 10, 10)
        plt.imshow(original_with_dots, cmap='gray')
        plt.title('Original with Dots')

        plt.show()

# Example usage
image_paths = ['/content/dr_test_1190_NV (1).jpg', '/content/img_02 (1).jpeg', '/content/img_04.jpeg',
    '/content/img_05.jpeg', '/content/img_06.jpeg', '/content/img_07.jpeg',
    '/content/img_08.jpeg', '/content/img_09 (1).jpeg', '/content/img_10.jpeg',
    '/content/img_11 (1).jpeg', '/content/img_15.jpeg', '/content/img_16.jpeg', '/content/img_17.jpeg',
    '/content/img_18.jpeg', '/content/img_19.jpeg', '/content/img_21 (1).jpeg',
    '/content/img_23.jpeg', '/content/img_24.jpeg', '/content/img_25.jpeg', '/content/Screenshot 2024-06-19 234819.png', '/content/Screenshot 2024-06-19 234832.png',
    '/content/Screenshot 2024-06-19 234844.png', '/content/Screenshot 2024-06-19 234853.png', '/content/Screenshot 2024-06-19 234900.png']
process_and_visualize_images(image_paths)
